### Description
This notebook presents "equivalent scheduled outage factors" by generator type, calculated using North American Electric Reliability Corporation (NERC) Generating Availability Data System (GADS) data.

Unit type definitions are consistent with [1], namely `CC` (codes 850-852), `CT` (code 300), `HD` (code 500), `NU` (code 200) and `ST` (codes 100 and 650). Only units of these unit types located within CONUS are considered in this analysis.

The equivalent scheduled outage factor includes all maintenance and planned outage and derating event types (event codes `MO`, `ME`, `PO`, `PE`, `D4`, `DM`, `DP`, and `PD`). See [2] for more information.

The starting point for this analysis is the creation of hourly time series of unit-level unavailable capacity from these event types [3].

With the unit-level time series created, we next calculate unit-level outage factors for each month of year. 

Next, we turn to calculating geography by unit-type averages, using the definitions mentioned above. We note the following data limitations:
* The data provides nameplate ranges for each unit rather than precise nameplate values
* The data does not provide a fully reliable field for determining the active period of each unit

We address each of these limitations in turn. First, we estimate unit nameplates from the ranges provided by NERC (e.g., using the midpoint of the range). Second, we estimate the active period of each unit at a monthly resolution by determining the date range of reporting to the GADS Events and GADS Performance tables.

With these steps complete, we calculate the final equivalent scheduled outage factors by unit type, both by NERC region and for all of CONUS [4]. We weight unit-level results by nameplate (e.g., a unit estimated at 200 MW will have twice the "weight" as a unit estimated at 100 MW) and by confirmed active period length (e.g., a unit confirmed active for 2 years will have twice the "weight" as a unit confirmed active for 1 year).

Finally, we do not report factors for any geography by unit type that doesn't contain at least five generators, consistent with NERC privacy protection requirements. Here, only TRE-NU is affected.

In [1]:
#imports
import pandas as pd

In [2]:
#equivalent scheduled outage factors -- NERC regions
regions_df = pd.read_csv("../data/outage_factors_region_type_2013_2023.csv", index_col=0)
regions_df

,01,02,03,04,05,06,07,08,09,10,11,12
MRO-CC,0.0561,0.0804,0.1807,0.2669,0.1604,0.0562,0.0200,0.0155,0.0911,0.3227,0.1989,0.0834
MRO-CT,0.0536,0.0547,0.0937,0.1187,0.1050,0.0522,0.0237,0.0214,0.0661,0.1330,0.1165,0.0774
MRO-HD,0.0813,0.0851,0.1376,0.1501,0.1232,0.0758,0.0502,0.0496,0.0989,0.1649,0.1468,0.0984
MRO-NU,0.0107,0.0285,0.0826,0.1700,0.1099,0.0392,0.0193,0.0185,0.0835,0.2658,0.1556,0.0407
MRO-ST,0.0747,0.0830,0.1786,0.2738,0.2239,0.0796,0.0339,0.0334,0.1092,0.2054,0.1818,0.0995
NPCC-CC,0.0282,0.0283,0.0987,0.1924,0.1239,0.0346,0.0148,0.0121,0.0696,0.1833,0.1243,0.0514
NPCC-CT,0.0595,0.0608,0.1063,0.1551,0.1035,0.0240,0.0100,0.0062,0.0516,0.1768,0.1333,0.0754
NPCC-HD,0.0676,0.0750,0.0999,0.1051,0.1004,0.0797,0.0632,0.0637,0.0820,0.1310,0.1173,0.0786
NPCC-NU,0.0079,0.0162,0.1049,0.1987,0.1027,0.0162,0.0046,0.0081,0.0332,0.1107,0.0397,0.0032
NPCC-ST,0.0769,0.0867,0.1633,0.2676,0.1914,0.0384,0.0237,0.0289,0.0963,0.2967,0.2580,0.1124


In [3]:
#equivalent scheduled outage factors -- CONUS
conus_df = pd.read_csv("../data/outage_factors_conus_type_2013_2023.csv", index_col=0)
conus_df

,01,02,03,04,05,06,07,08,09,10,11,12
CONUS-CC,0.0496,0.0696,0.1774,0.2333,0.1545,0.0452,0.0181,0.0151,0.0570,0.1970,0.1763,0.0766
CONUS-CT,0.0446,0.0545,0.1017,0.1269,0.0932,0.0346,0.0164,0.0155,0.0538,0.1365,0.1180,0.0718
CONUS-HD,0.1169,0.1233,0.1378,0.1313,0.1101,0.0888,0.0849,0.0981,0.1363,0.1834,0.1696,0.1291
CONUS-NU,0.0134,0.0492,0.1184,0.1670,0.1041,0.0293,0.0092,0.0114,0.0564,0.1555,0.0982,0.0285
CONUS-ST,0.0735,0.0988,0.2053,0.2649,0.1982,0.0722,0.0374,0.0394,0.1039,0.2194,0.2027,0.1090


### References

[1] Murphy, S., Apt, J., Moura, J., and Sowell, F. (2018). Resource adequacy risks to the bulk power system in North America. Applied Energy. https://doi.org/10.1016/j.apenergy.2017.12.097

[2] North American Electric Reliability Corporation (2024). Generating Availability Data System Data Reporting Information. https://www.nerc.com/pa/RAPA/gads/pages/data%20reporting%20instructions.aspx

[3] Murphy, S. Time series generation from NERC GADS data. https://github.nrel.gov/smurphy2/NERC-GADS

[4] Murphy, S. Outage factor calculation from NERC GADS data. https://github.nrel.gov/smurphy2/NERC-GADS-outage-factors
